In [ ]:
import requests #librería estándar de Python para manejar peticiones HTTP de forma sencilla
import pandas as pd #para estructurarlos
import numpy as np #trabaja con arrays numéricos de alta eficiencia y para el manejo avanzado de valores nulos

Fase 1: Extracción de datos de películas

Objetivo: Extraer 100 películas del endpoint

In [14]:

# 1. Definir el endpoint de la API
url_api = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

# 2. Realizar la solicitud HTTP
try:
    response = requests.get(url_api) #petición GET porque solo necesitas obtener datos del servidor (la API).
    response.raise_for_status() #comprueba el código de estado HTTP (200 OK, 404 Not Found, 500 Server Error, etc.) 
    #y lanza una excepción si el código indica un fallo (4xx o 5xx).
    data = response.json()
except requests.exceptions.RequestException as e:
    print(f"Error al obtener los datos de la API: {e}") #asegura que el programa no colapse si hay fallos de red
    data = []

# 3. Almacenar los datos en un DataFrame (DataFrame)

if data: #el programa solo continúa si realmente recibió datos de la API.
    # La API devuelve una lista de diccionarios (convirtiendo) el texto plano de la respuesta de la API (que está en formato JSON) 
    # a una estructura de datos nativa de Python (lista de diccionarios).
    df_peliculas = pd.DataFrame(data)
    #conviertes rápidamente la lista de diccionarios (estructura ideal de JSON) en un objeto DataFrame de Pandas, 
    # la estructura fundamental para el análisis de datos en Python

    # Solo nos interesan las primeras 100 películas: filtrado o control de volumen
    df_peliculas = df_peliculas.head(100) 

    # 4. Seleccionar y renombrar las columnas según el enunciado
    # Las columnas del enunciado son: Título, Año de lanzamiento, Duración (en minutos), Género, Contenido para adultos
    
    df_peliculas = df_peliculas.rename(columns={ #paso crucial de Estandarización
        'titulo': 'titulo',
        'año': 'ano_lanzamiento',
        'duracion': 'duracion_minutos',
        'genero': 'genero',
        'adultos': 'contenido_adulto'
    })

    print("Extracción y preparación de datos completada.")
    print(df_peliculas.head())
else:
    print("No se pudieron obtener datos para el DataFrame.")

# Seleccionamos solo las columnas que necesitamos (ya que nuestro API tiene más)
df_peliculas = df_peliculas[['titulo', 'ano_lanzamiento', 'duracion_minutos', 'genero', 'contenido_adulto']]


Extracción y preparación de datos completada.
   id                 titulo  ano_lanzamiento  duracion_minutos  genero  \
0   1          The Godfather             1972               175  Crimen   
1   2  The Godfather Part II             1974               202  Crimen   
2   3           Pulp Fiction             1994               154  Crimen   
3   4           Forrest Gump             1994               142   Drama   
4   5        The Dark Knight             2008               152  Acción   

   contenido_adulto    subtitulos  
0             False      [es, en]  
1             False      [es, en]  
2              True      [es, en]  
3             False  [es, en, fr]  
4             False      [es, en]  


In [ ]:
df_clean = df_peliculas.replace({np.nan: None, 'nan': None, 'NaN': None})
#Se estandariza la representación de valores nulos, convirtiendo todas las formas de NaN (numérico y de cadena) al objeto None de Python, asegurando la correcta inserción de valores 
# NULL en la base de datos MySQL y previniendo errores de integridad de datos.

In [15]:
data

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [11]:
df_peliculas

,titulo,ano_lanzamiento,duracion_minutos,genero,contenido_adulto
0,The Godfather,1972,175,Crimen,False
1,The Godfather Part II,1974,202,Crimen,False
2,Pulp Fiction,1994,154,Crimen,True
3,Forrest Gump,1994,142,Drama,False
4,The Dark Knight,2008,152,Acción,False
...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False
96,Requiem for a Dream,2000,102,Drama,True
97,Memento,2000,113,Thriller,True
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False


Fase 2: Creación de la base de datos e inserción de datos

In [12]:
pip install mysql-connector-python pandas

In [ ]:
import mysql.connector #Carga la librería que permite a Python comunicarse con el servidor MySQL
from mysql.connector import errorcode #Importa módulos para identificar y manejar errores específicos de MySQL
from mysql.connector import Error # <-- Se necesita para el primer except

# 1. Configuración de la conexión 
nombre_bd = "Adalab_pelis"
cnx = None # Inicializamos la variable de conexión

try:
    # Intento de conexión inicial sin especificar la DB para crearla
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
    )
    print('Conexión exitosa para crear DB')
    
    mycursor = cnx.cursor() #Crea un objeto cursor a través del cual se enviarán los comandos SQL al servidor MySQL
    query = f"CREATE DATABASE IF NOT EXISTS {nombre_bd}" #para crear la base de datos
    mycursor.execute(query) #Envía la sentencia CREATE DATABASE al servidor MySQL
    print("Base de datos creada/asegurada.")
    
    # Cerramos la conexión inicial (sin DB) temporalmente
    mycursor.close()
    cnx.close()
    
    # 2. Conexión a la Base de Datos Creada 
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
        database=nombre_bd # <-- Conectamos directamente a la DB
    )
    mycursor = cnx.cursor()

    # 4. Creación de la Tabla 'peliculas'
    TABLES = {}
    TABLES['peliculas'] = ( #Define la estructura de la tabla peliculas en una cadena de texto
        "CREATE TABLE `peliculas` ("
        "  `id_pelicula` INT NOT NULL AUTO_INCREMENT,"
        "  `titulo` VARCHAR(255) NOT NULL,"
        "  `ano_lanzamiento` YEAR NOT NULL,"
        "  `duracion_minutos` INT NOT NULL,"
        "  `genero` VARCHAR(100) NOT NULL,"
        "  `contenido_adulto` VARCHAR(3) NOT NULL,"
        "  PRIMARY KEY (`id_pelicula`)"
        ") ENGINE=InnoDB")

    try:
        print("Creando tabla peliculas...")
        mycursor.execute(TABLES['peliculas']) #Ejecuta el comando SQL para crear la tabla
    except mysql.connector.Error as err: #Si falla la creación, el código comprueba si el error es el código
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR: #Si la tabla ya existe, simplemente se imprime un mensaje y el script continúa
            print("La tabla 'peliculas' ya existe, continuando...")
        else:
            print(f"Error al crear la tabla: {err}") #Si el error es grave, se imprime el error para debuggear
    
    # 5. Inserción de Datos desde el DataFrame (df_peliculas)
    
    # Preparamos la sentencia SQL de inserción
    insert_query = ( #Define la plantilla INSERT con los marcadores de posición %s. Importante orden de inserción
        "INSERT INTO peliculas "
        "(titulo, ano_lanzamiento, duracion_minutos, genero, contenido_adulto) "
        "VALUES (%s, %s, %s, %s, %s)"
    )
    
    # Convertimos el DataFrame a una lista de tuplas para la inserción
    records_to_insert = [ #Usa una comprensión de listas para iterar sobre el DataFrame de Pandas y convertir cada fila en una tupla de Python
        (row['titulo'], row['ano_lanzamiento'], row['duracion_minutos'], row['genero'], row['contenido_adulto'])
        for index, row in df_peliculas.iterrows()
    ]
    
    # Ejecutamos la inserción
    mycursor.executemany(insert_query, records_to_insert) #envía todos los datos en un solo bloque al servidor
    
    # Confirmar los cambios
    cnx.commit() #hace los cambios permanentes en la base de datos
    print(f"Inserción completada. {mycursor.rowcount} filas insertadas.")

except Error as e: # <-- Maneja errores de la primera conexión y creación de DB
    print(f"Error general en la conexión/DB: {e}")
    
finally: #El bloque finally siempre se ejecuta, independientemente de si hubo un error o no
    if cnx and cnx.is_connected():
        mycursor.close() #Cierra la conexión y el cursor, liberando los recursos del servidor
        cnx.close()
        print("Conexión a MySQL cerrada.")

Conexión exitosa para crear DB
Base de datos creada/asegurada.
Creando tabla peliculas...
La tabla 'peliculas' ya existe, continuando...
Inserción completada. 100 filas insertadas.
Conexión a MySQL cerrada.
